In [2]:
import tensorflow as tf
import os
import math
import time
import pickle
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
CIFAR_DIR = "./cifar-10-batches-py/"
#print (os.listdir(CIFAR_DIR))
#bat_data, bat_labels = cifar10_input.distorted_inputs(data_dir=CIFAR_DIR,batch_size=20)
#print(bat_data)

In [3]:
def load_data(filename):
    """read data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='bytes')
        #print(data[b'data'][1])
        #print(data.shape)
        return data[b'data'],data[b'labels']
def weight_variable(shape,stddev):
    initial = tf.truncated_normal(shape, stddev=stddev)
    return tf.Variable(initial)
def bias_variable(shape,value):
    initial = tf.constant(value=value,shape=shape)
    return tf.Variable(initial)
def conv2d(x, W): 
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x,name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME',name = name)

def norm_lrn(x,name):
    return tf.nn.lrn(x, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,name=name)
def loss_acc_curve(step,loss,acc,num):
    #step = np.linspace(step[0], step[num],  100) 
    #loss = np.linspace(step[0], loss[num], 100)    
    #acc = np.linspace(step[0], acc[num], 100)    
    plt.plot(step, loss,'-',label="loss")
    plt.plot(step, acc,'-',label="acc")

    plt.title('loss_acc')
    plt.xlabel('step')
    plt.ylabel('loss_acc')
    plt.savefig('./loss_acc/loss_acc_qqq',dpi=600)  
    plt.show()
    
    plt.plot(acc, loss,'-',label="loss-acc")
    plt.title('loss-acc')
    plt.xlabel('acc')
    plt.ylabel('loss')
    plt.savefig('./loss_acc/loss-acc_qqq',dpi=600)  
    plt.show()
    
    

In [4]:
def sort_image_real(num):
    if num == 1:
        x = '小汽车'
    elif num == 2:
        x = '鸟类'
    elif num == 3:
        x = '猫'
    elif num == 4:
        x = '鹿'
    elif num == 5:
        x = '狗'
    elif num == 6:
        x = '青蛙'
    elif num == 7:
        x = '马'
    elif num == 8:
        x = '船'
    elif num ==9:
        x = '卡车'
    elif num == 0:
        x = '飞机'
    else :
        x = 'error'
    return x
        


In [20]:
def distort(x_batch,batch_size,height,width):
    #(128,3072)
    image = []
    #print(x)
    #x = tf.convert_to_tensor(x_array)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(0,batch_size):
            y = x_batch[i].reshape((3,height,width)) 
            y = y.transpose((1,2,0))#32 32 3
            
            distorted_image = tf.image.random_flip_left_right(y)
            distorted_image = tf.image.random_brightness(distorted_image,
                                                        max_delta=63)
            distorted_image = tf.image.random_contrast(distorted_image,
                                                       lower=0.2, upper=1.8)
            distorted_image = tf.image.per_image_standardization(distorted_image)
            
            distorted_image_ = distorted_image.eval()
            image.extend(distorted_image_)
    return np.vstack(image)

In [6]:
for i in range(0,5):
    print(i)

0
1
2
3
4


In [8]:
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        
        for filename in filenames:
            data,labels = load_data(filename)
            print(data.shape)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        #print(all_data[1])
        #print(data[1])
        self._image = self._data
        #self._data = self._data / 127.5 -1
        
        self._labels = np.hstack(all_labels)
       
        '''print (self._data.shape)
        print (self._labels.shape)
        '''
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        #[0,1,2,3,4,5]-> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
        
    def next_batch(self,batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples  ")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_image = self._image[self._indicator: end_indicator]
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        
        
        self._indicator = end_indicator
        return batch_image,batch_data,batch_labels

                            



In [9]:
train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d.bin' % i) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch.bin')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,False)

(10000, 3072)
(10000, 3072)
(10000, 3072)
(10000, 3072)
(10000, 3072)
(10000, 3072)


In [18]:
batch_image,batch_data, batch_labels = train_data.next_batch(20)

print(batch_data)


[[245 233 224 ...  44  38  37]
 [ 78  78  77 ... 116 115 115]
 [ 94  96  96 ...  81  75  71]
 ...
 [126 119 115 ... 163 163 160]
 [116 147 151 ... 111  90 118]
 [144 154 152 ... 201 195 190]]


In [22]:
a = distort(batch_data,20,32,32)
print(a.shape)

(20480, 3)


In [25]:
batch_size = 20
x = tf.placeholder(tf.float32,[batch_size*32*32,3])
print(x)
x_image = tf.reshape(x, [batch_size,32,32,3])
rate  = tf.placeholder(tf.float32)
y = tf.placeholder(tf.int64,[None])
#[CONV->POOL]
W_conv1_1 = weight_variable([5, 5, 3, 64],0.05)
b_conv1_1 = bias_variable([64],0.0)
bias_add1 = tf.nn.bias_add(conv2d(x_image, W_conv1_1) ,b_conv1_1)
h_conv1_1 = tf.nn.relu(bias_add1,name = "h_conv1_1") 
h_pool1 = max_pool_2x2(h_conv1_1,"h_pool1")
norm1 = norm_lrn(h_pool1,"norm1")
#[CONV->POOL]
W_conv2= weight_variable([3,3, 64, 64],0.05)
b_conv2 = bias_variable([64],0.1)
bias_add2_1 = tf.nn.bias_add(conv2d(norm1, W_conv2) ,b_conv2)
h_conv2_1 = tf.nn.relu(bias_add2_1,name = "h_conv2_1")
bias_add2_2 = tf.nn.bias_add(conv2d(h_conv2_1, W_conv2) ,b_conv2)
h_conv2_2 = tf.nn.relu(bias_add2_2,name = "h_conv2_2")
norm2 = norm_lrn(h_conv2_2,"norm2")
h_pool2 = max_pool_2x2(norm2,"h_pool2")

#[FC1]
print (h_pool2)
norm_flat = tf.reshape(h_pool2, [batch_size, -1])
print(norm_flat)
dim = norm_flat.get_shape()[1].value
print(dim)
W_fc1 = weight_variable([dim,1024],0.04)
b_fc1 = bias_variable([1024],0.1)
h_fc1 = tf.nn.relu(tf.matmul(norm_flat, W_fc1) + b_fc1,name ="h_fc1")
print(h_fc1)
#[FC2]
W_fc2 = weight_variable([1024,128],0.04)
b_fc2 = bias_variable([128],0.1)
h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2,name ="h_fc2")
#[outpout]
W_fc3 = weight_variable([128,10],1/128.0)
b_fc3 = bias_variable([10],0.1)
y_ = tf.add(tf.matmul(h_fc2, W_fc3), b_fc3, name="y_")

print(x_image)
print(y_)
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
#y_ -> sofmax
#y -> one_hot
#loss = ylogy_
#bool
image_prediction = tf.argmax(y_,1)
# [1,#0,1,1,1,0,0,0]
correct_prediction = tf.equal(image_prediction,y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))
top_k_op = tf.nn.in_top_k(y_, y, 1)
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    ema = tf.train.ExponentialMovingAverage(decay=0.9999)
    with tf.control_dependencies([train_op]):
        optimize_op = ema.apply(tf.trainable_variables())


Tensor("Placeholder_6:0", shape=(20480, 3), dtype=float32)
Tensor("h_pool2_2:0", shape=(20, 8, 8, 64), dtype=float32)
Tensor("Reshape_5:0", shape=(20, 4096), dtype=float32)
4096
Tensor("h_fc1_2:0", shape=(20, 1024), dtype=float32)
Tensor("Reshape_4:0", shape=(20, 32, 32, 3), dtype=float32)
Tensor("y__2:0", shape=(20, 10), dtype=float32)


In [ ]:
init = tf.global_variables_initializer()
train_steps = 10000
test_steps = 50
acc_val_new = 0
loss_ = []
acc_ = []
step_ = []
num = 0
saver = tf.train.Saver()
#test_batch_data, test_batch_labels = cifar10_input.inputs(eval_data=True, data_dir=CIFAR_DIR, batch_size=batch_size)
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        start_time = time.time()
        #batch_data, batch_labels = cifar10_input.distorted_inputs(data_dir=CIFAR_DIR,batch_size=batch_size)
        train_batch_image,train_batch_data, train_batch_labels = train_data.next_batch(batch_size)
        train_batch_data_ = distort(train_batch_data,batch_size,32,32)
        loss_val, acc_val,_ = sess.run(
            [loss,accuracy,train_op],feed_dict={
                x: train_batch_data_,y: train_batch_labels})
        duration = time.time() - start_time
        if (i+1) % 100 == 0:
            loss_.append(loss_val)
            acc_.append(acc_val)
            step_.append(i)
            examples_per_sec = batch_size / duration
            sec_per_batch = float(duration)
            print('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch),acc: %4.5f'% 
                  (i+1, loss_val, examples_per_sec, sec_per_batch,acc_val))
        if acc_val > acc_val_new:
            acc_val_new = acc_val
            saver.save(sess, './model/my-model',write_meta_graph=False)
            if acc_val_new >=0.999:
                print(format_str % (i+1, loss_val, examples_per_sec, sec_per_batch,acc_val))
                num = i
                break
        num = i
        
    loss_acc_curve(step_,loss_,acc_,num)
    all_test_acc_val = []
    for j in range(test_steps):
        test_batch_image,test_batch_data, test_batch_labels  = test_data.next_batch(batch_size)
        #print(test_batch_data.shape)
        
        test_acc_val = sess.run(
            accuracy,feed_dict = {
                x: test_batch_data,y: test_batch_labels})
        sort_prediction = sess.run(
            image_prediction,feed_dict = {
                x: test_batch_data,y: test_batch_labels})
        sort_real = sess.run(
            y,feed_dict = {
                x: test_batch_data,y: test_batch_labels})
        
        image= test_batch_image[0].reshape((3,32,32)) #32 32 3
        image = image.transpose((1,2,0))
        plt.imshow(image)
        plt.show()
        print('acc: %4.5f'%test_acc_val)
        #sort_prediction = tf.nn.top_k(sort_prediction,1)
        #sort_real = tf.nn.top_k(sort_real,1)
        print('[Test] Step:%d 预测分类是 %s，真实分类是 %s'% 
              (j+1 ,sort_image_real(sort_prediction[0]),sort_image_real(sort_real[0])))
        all_test_acc_val.append(test_acc_val)
    test_acc = np.mean(all_test_acc_val)
    print(' acc: %4.5f'% (test_acc))
          
 

In [ ]:
#step 2400, loss=0.66 (95.6 examples/sec; 1.339 sec/batch),acc: 0.75000   


"""    
   all_test_acc_val = []
    for j in range(test_steps):
        test_batch_data, test_batch_labels  = test_data.next_batch(batch_size)
        test_acc_val = sess.run(accuracy,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        #image_prediction = sess.run(y_,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        #image_real = sess.run(y,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        #image_prediction = tf.nn.top_k(image_prediction,1)
        #image_real = tf.nn.top_k(image_real,1)        
        all_test_acc_val.append(test_acc_val)
    test_acc = np.mean(all_test_acc_val)
    #print('[Test] Step:%d 真实分类是 %s，预测分类是 %s, acc: %4.5f'% (i+1 ,image_prediction.values,image_real.values,test_acc))
    print('[Test] Step:%d , acc: %4.5f'% (i+1 ,test_acc))
     
"""    


In [ ]:
"""
num_examples = 300
    all_test_acc_val = []
    num_inter = int(math.ceil(num_examples / batch_size))
    true_count = 0   
    total_sample_count = num_inter * batch_size
    step = 0
    while step < num_inter:
        #test_batch_data, test_batch_labels= sess.run([test_batch_data, test_batch_labels]   ) 
        #test_batch_data = tf.reshape(test_batch_data,[20,1728]
        test_batch_data, test_batch_labels  = test_data.next_batch(batch_size)
        test_acc_val,count_pred_true =sess.run(
            [accuracy,top_k_op],
            feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        sort_prediction = sess.run(
            image_prediction,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        sort_real = sess.run(
            y,feed_dict = {x: test_batch_data,y: test_batch_labels,keep_prob:1.0})
        true_count += np.sum(count_pred_true)
        #print(image_real)
        #print(sess.run(image_prediction.indices))
        sort_prediction = tf.nn.top_k(sort_prediction,1)
        sort_real = tf.nn.top_k(sort_real,1)
        print('[Test] Step:%d 预测分类是 %s，真实分类是 %s'% 
              (step ,sess.run(sort_prediction.indices),sess.run(sort_real.indices)))
        step += 1
    prediction = true_count / total_sample_count
    print ('precision @ 1 = %.3f,acc: %4.5f '
           % (prediction,test_acc_val))
"""